In [5]:
# import libaries 
import cv2
import numpy as np
import os
# from google.colab.patches import cv2_imshow

# read the video using VideoCapture funtion
cap = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")

try:
    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imshow("inter", frame) 
        #   cv2_imshow(frame) # for google colab only

        # esc key = 27
        if cv2.waitKey(40) == 27:
            break

    cv2.destroyAllWindows()
    cap.release

except:
    cv2.destroyAllWindows()
    cap.release


In [3]:
os.getcwd()

'd:\\User\\Coding_Projects\\Python Projects\\Motion-emotion-detection'

In [1]:
# Setting GPU-enabled Tensorflow
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      # tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      # tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      tf.config.experimental.set_memory_growth(gpu, True)
      print("sucessful")
  except RuntimeError as e:
    print(e)

sucessful


In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [1]:
# disable gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import cv2
from deepface import DeepFace
import numpy as np
# import tensorflow as tf
# # face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
# print(len(tf.config.list_physical_devices('GPU'))>0)

def analyse_face():
    # imagepath = os.getcwd() + "/data/happy_face_woman.png"
    imagepath = os.getcwd() + "/data/face.png"
    # imagepath = os.getcwd() + "/data/class.png"
    image = cv2.imread(imagepath)
    # face_analysis = DeepFace.analyze(image)
    face_analysis = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
    print(face_analysis)

analyse_face()

1/1 [==============================] - 4s 4s/step
{'emotion': {'angry': 27.076569199562073, 'disgust': 0.00011799746744145523, 'fear': 59.964632987976074, 'happy': 12.385643273591995, 'sad': 0.5619456525892019, 'surprise': 0.0007821342478564475, 'neutral': 0.010309615026926622}, 'dominant_emotion': 'fear', 'region': {'x': 0, 'y': 0, 'w': 87, 'h': 92}}


# (image) face + emotion detection

In [21]:
import cv2
from deepface import DeepFace
import numpy as np
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

filename = "neutral.png"
# filename = "happy.png"
# filename = "sad.png"

imagepath = os.getcwd() + "/data/" + filename
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5)
for (x,y,w,h) in faces:
    temp = (x,y,w,h)
    image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    color = (255,0,0)
    try:
        analyze = DeepFace.analyze(image[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
        emotion_label = analyze['dominant_emotion']
        state_of_mind_label = get_state_of_mind(analyze['emotion'])
        image=cv2.putText(image, emotion_label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        image=cv2.putText(image, state_of_mind_label, (x, y + h + 12 ),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    except Exception as e:
        print(e)

cv2.imshow("image", image) 
cv2.imwrite(os.getcwd() + "/output/" + filename , image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step


In [58]:
analyze['emotion']

{'angry': 4.505497485726922,
 'disgust': 0.00010644097935775428,
 'fear': 85.11115819878329,
 'happy': 8.703607839560975,
 'sad': 1.666227082385247,
 'surprise': 0.0012948668744324054,
 'neutral': 0.012110988603768191}

# State of mind Algorithm

In [3]:
# pass in a dictionary of confidence score for each emotion
def get_state_of_mind(emotions_dict):
    # declaring variables    
    confusion_list = ['neutral', 'surprise'] # duplicated combinatorial emotions for 'sad' with 'Disappointment/dissatisfaction', so 'sad' is removed from 'Confusion'
    satisfaction_delighterd_list = ['happy', 'neutral']
    disappointment_disstisfaction_list = ['neutral', 'sad']
    frustrated_list = ['sad', 'angry', 'neutral']
    # dictionary of ecombinatorial emotions with its respective state of mind
    state_of_mind_dict = {'Confusion': confusion_list, 
    'Satisfaction/Delighted': satisfaction_delighterd_list, 
    'Disappointment/Dissatisfaction': disappointment_disstisfaction_list,
    'Frustrated': frustrated_list}

    # dictionary used to store and determin the state of mind based on the combinatorial emotions
    state_of_mind_count = {'Confusion': [0, 0], 
    'Satisfaction/Delighted': [0, 0], 
    'Disappointment/Dissatisfaction': [0, 0],
    'Frustrated': [0, 0, 0]}
    state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
    count = 0
    state_switched = False

    # sort emotion from highest confidence score to lowest
    # sorted_emotions = emotions_dict # TODO: use for testing
    sorted_emotions = sorted(emotions_dict, key=emotions_dict.get, reverse=True)[:]
    for emotion in sorted_emotions:
        for state in state_of_mind_list:
            # if the emotion belongs to a certain state of mind
            if emotion in state_of_mind_dict[state]: 
                if count == 0:
                    state_of_mind_list.remove(state)
                    state_of_mind_list.insert(0, state)
                    state_switched = True
                # get the index of the emotion in the dictionary
                idx = state_of_mind_dict[state].index(emotion)
                # set the count to 1
                state_of_mind_count[state][idx] = 1
            
            # check if all emotions of a certain state of mind were matched
            if sum(state_of_mind_count[state]) == len(state_of_mind_count[state]):
                return state
        
        if state_switched:
            count = 1

In [20]:
# test case for state of mind algorithm
testcase1 = ['fear', 'neutral', 'angry', 'surprise', 'happy', 'sad', 'disgust'] #neutral + suprise = confusion
testcase2 = ['fear', 'happy', 'angry', 'sad', 'neutral', 'surprise', 'disgust'] #Happy + Neutral = Satisfaction / Delighted
testcase3 = ['fear', 'sad', 'neutral', 'disgust', 'angry', 'surprise', 'happy'] #Neutral + Sadness = Disappointment / Dissatisfaction
testcase4 = ['fear', 'sad', 'angry', 'happy', 'neutral', 'surprise', 'disgust'] #Sad + Angry + Neutral = Frustrated
testcase5 = ['fear', 'happy', 'angry', 'sad', 'neutral', 'surprise', 'disgust']

testcase_all = [testcase1, testcase2, testcase3, testcase4, testcase5]

# get_state_of_mind(analyze['emotion'])

# for x in range(len(testcase_all)):
#     print("case ", x+1)
#     get_state_of_mind(testcase_all[x])

s = get_state_of_mind(testcase4)

{'Confusion': [0, 0], 'Satisfaction/Delighted': [1, 0], 'Disappointment/Dissatisfaction': [0, 1], 'Frustrated': [1, 1, 1]}
Frustrated
matched state of mind!


In [60]:
state_of_mind_count

{'Confusion': [1, 1],
 'Satisfaction/Delighted': [1, 1],
 'Disappointment/Dissatisfaction': [1, 0, 0],
 'Frustrated': [1, 1, 1]}

In [50]:
state_of_mind_count['Disappointment/Dissatisfaction'][1] = 999

In [38]:
sorted_emotions = ['fear', 'happy', 'angry', 'sad', 'neutral', 'surprise', 'disgust']

In [45]:
state_of_mind_count

{'Confusion': [0, 0],
 'Satisfaction/Delighted': [0, 0],
 'Disappointment/Dissatisfaction': [0, 0, 0],
 'Frustrated': [0, 0, 0]}

# video face detection

In [23]:
import cv2
from deepface import DeepFace
import numpy as np
import os

filename = 'video_1'
# filename = 'video_2'

# declare the facce detection model to detect human faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# read the video using VideoCapture funtion
# video = cv2.VideoCapture(0)
video = cv2.VideoCapture(os.getcwd() + "/data/" + filename + ".mp4")

# get width and height
frame_width = int(video.get(3))
frame_height = int(video.get(4))

# declare a VideoWriter instance
result_mp4 = cv2.VideoWriter(os.getcwd() + "/output/" + filename + '_result.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 10, (frame_width, frame_height))

try:
    while video.isOpened():
        # read the video into frame
        ret, frame = video.read()
        ret, frame1 = video.read() 

        # A. face detection, emotion classification, state of mind
        # A1. convert the frame to grayscale for CascadeClassifier
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # A2. find coordinates of faces, returns (x,y,w,h)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5)

        # B. motion detection
        # B1. find the absolute difference between current frame and previous frame
        diff = cv2.absdiff(frame, frame1)
        # B2. convert the frame difference to grayscale 
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        # B3. apply gaussian smoothing onto the grayscale frame to reduce noise 
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        # B4. apply thresholding 
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        # B5. dilate the frame to remove small unwanted detection
        dilated = cv2.dilate(thresh, None, iterations=3)
        # B6. get the contours of detected movement in frame
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # legends in video
        cv2.putText(frame, "Blue: Emotion", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(frame, "Red: State of Mind", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, "Green: Motion", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # loop through all detected faces
        for (x,y,w,h) in faces:
            color = (255,0,0) # blue color
            # A3. draw a blue rectangle around the face
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            try:
                # A4. perform emotion classification on the face detected
                analyze = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
                # A5. get the dominant emotion as the label 
                emotion_label = analyze['dominant_emotion']
                # A6. perform state of mind calculation
                state_of_mind_label = get_state_of_mind(analyze['emotion'])
                # A7. write the emotion on the frame in blue text
                frame=cv2.putText(frame, emotion_label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
                # A8. write the state of mind on the frame in red text
                frame=cv2.putText(frame, state_of_mind_label, (x, y + h + 12 ),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            except Exception as e:
                print(e)

        # loop through all contours
        for contour in contours:
            # B7. generate x, y coordinate, width, height based on the contour
            (x, y, w, h) = cv2.boundingRect(contour)
            # B8. draw a green rectangle if the contour area is greater than 600
            if cv2.contourArea(contour) > 600:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        result_mp4.write(frame)

        cv2.imshow("video", frame) 
        frame = frame1
        ret, frame1 = video.read()
        #   cv2_imshow(frame) # for google colab only
        # cv2.waitKey(5000)
        if cv2.waitKey(40) == 27:
            break

    cv2.destroyAllWindows()
    video.release
    result_mp4.release()


except Exception as e:
    print(e)
cv2.destroyAllWindows()
video.release
result_mp4.release()


OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



In [1]:
import cv2

# read the video and extract info about it
# cap = cv2.VideoCapture('vid.avi')
cap = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")


# get total number of frames and generate a list with each 30 th frame 
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
x = [i for i in range (1, totalFrames) if divmod(i, int(30))[1]==0]


for myFrameNumber in x:
    cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)
    while True:
        ret, frame = cap.read()
        cv2.imshow("video", frame)

        ch = 0xFF & cv2.waitKey(1000)
        if ch == 27:
            break
cv2.destroyAllWindows()

In [3]:
x

[30, 60, 90, 120, 150]

In [5]:
!pip show tensorflow

Name: tensorflow
Version: 2.9.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\guang\anaconda3\envs\deepfacepy37\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: deepface, retina-face


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [1]:
!nvidia-smi

Mon Nov 14 22:51:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.83       Driver Version: 451.83       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:5A:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [6]:
!nvidia-smi

Thu Nov 10 01:44:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.83       Driver Version: 451.83       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:5A:00.0 Off |                  N/A |
| N/A   48C    P8    N/A /  N/A |   1432MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  